In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Loading the dataset
data = pd.read_excel("data.xlsx")

# Selecting relevant columns for loyalty-based segmentation
loyalty_data = data[['PreviousPurchases', 'LoyaltyPoints']].copy()

# Normalizing the continuous variables
scaler = StandardScaler()
loyalty_data[['PreviousPurchases', 'LoyaltyPoints']] = scaler.fit_transform(loyalty_data[['PreviousPurchases', 'LoyaltyPoints']])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Using silhouette scores to determine the optimal number of clusters
silhouette_scores = []
for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(loyalty_data)
    silhouette_scores.append(silhouette_score(loyalty_data, kmeans.labels_))

# Plotting silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(range(2, 11), silhouette_scores, marker='o', linestyle='--')
plt.title('Silhouette Scores for Optimal Number of Clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
# Defining the optimal number of clusters based on the silhouette score (0.41 the highest, for 4 clusters)
optimal_clusters = 4  

kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
data['Cluster'] = kmeans.fit_predict(loyalty_data)

# Analyzing the clusters
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=['PreviousPurchases', 'LoyaltyPoints'])
cluster_centers['Cluster'] = range(optimal_clusters)
print(cluster_centers)

In [ ]:
# Grouping by clusters to analyze the segments
segment_analysis = data.groupby('Cluster')[['PreviousPurchases', 'LoyaltyPoints']].mean()
print(segment_analysis)

In [ ]:
# Creating marketing lists for email campaigns considering clusters' means
high_loyalty_customers = data[data['Cluster'] == 0]  
high_frequency_customers = data[data['Cluster'] == 3]
medium_loyalty_customers = data[data['Cluster'] == 2]  
low_loyalty_customers = data[data['Cluster'] == 1]  

# Exporting these lists to CSV files for use in marketing campaigns
high_loyalty_customers.to_csv('high_loyalty_customers.csv', index=False)
high_frequency_customers.to_csv('high_frequency_customers.csv', index=False)
medium_loyalty_customers.to_csv('medium_loyalty_customers.csv', index=False)
low_loyalty_customers.to_csv('low_loyalty_customers.csv', index=False)